In [2]:
!pip install torch torchaudio transformers librosa faiss-cpu tqdm


Defaulting to user installation because normal site-packages is not writeable


In [4]:
import torch
print("GPU available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


GPU available: True
Device: NVIDIA GeForce GTX 1070


In [8]:
import os

# Change this if needed
AUDIO_FOLDER = "/home/ivan/PycharmProjects/MPr/audio_samples/fma_small"

mp3_files = []
for root, _, files in os.walk(AUDIO_FOLDER):
    for file in files:
        if file.endswith(".mp3"):
            mp3_files.append(os.path.join(root, file))

# Optional: limit to the first 300 for now
mp3_files = mp3_files[:300]

print(f"Loaded {len(mp3_files)} files.")


Loaded 300 files.


In [9]:
import torch
from transformers import ClapProcessor, ClapModel

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = ClapProcessor.from_pretrained("laion/clap-htsat-unfused")
model = ClapModel.from_pretrained("laion/clap-htsat-unfused").to(device)


In [25]:
def embed_audio(filepath):
    try:
        waveform, sr = torchaudio.load(filepath)
        print(f"📂 Loaded '{filepath}' — shape: {waveform.shape}, sample rate: {sr}")

        waveform = waveform.mean(dim=0).unsqueeze(0)  # mono

        if sr != 48000:
            print(f"🔁 Resampling from {sr} → 48000 Hz")
            waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=48000)(waveform)

        # ✂️ Trim to exactly 30s = 1,440,000 samples
        max_len = 48000 * 30
        if waveform.shape[1] > max_len:
            print(f"✂️ Trimming waveform to 30s: {waveform.shape[1]} → {max_len}")
            waveform = waveform[:, :max_len]
        elif waveform.shape[1] < max_len:
            print(f"📏 Padding waveform to 30s: {waveform.shape[1]} → {max_len}")
            pad_len = max_len - waveform.shape[1]
            waveform = torch.nn.functional.pad(waveform, (0, pad_len))

        inputs = processor(
            audios=waveform.squeeze(0).cpu().numpy(),  # Convert to 1D NumPy array
            sampling_rate=48000,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            embedding = model.get_audio_features(**inputs)

        print(f"✅ Embedded {filepath}, vector shape: {embedding.shape}")
        return embedding.cpu().numpy().flatten()

    except Exception as e:
        print(f"❌ ERROR embedding {filepath}:\n{e}")
        traceback.print_exc()
        raise


In [26]:
embeddings = []
filenames = []
failures = []

for path in tqdm(mp3_files, desc="Embedding audio files"):
    try:
        vec = embed_audio(path)
        embeddings.append(vec)
        filenames.append(path)
    except Exception as e:
        failures.append((path, str(e)))

if embeddings:
    embeddings = np.vstack(embeddings)
    print(f"\n✅ Embedded {len(embeddings)} tracks.")
else:
    print("❌ No embeddings succeeded.")

print(f"❌ Failed: {len(failures)}")


Embedding audio files:   0%|                            | 0/300 [00:00<?, ?it/s]

📂 Loaded '/home/ivan/PycharmProjects/MPr/audio_samples/fma_small/115/115761.mp3' — shape: torch.Size([2, 1323119]), sample rate: 44100
🔁 Resampling from 44100 → 48000 Hz
✂️ Trimming waveform to 30s: 1440130 → 1440000


Embedding audio files:   1%|▏                   | 3/300 [00:00<01:10,  4.23it/s]

✅ Embedded /home/ivan/PycharmProjects/MPr/audio_samples/fma_small/115/115761.mp3, vector shape: torch.Size([1, 512])
📂 Loaded '/home/ivan/PycharmProjects/MPr/audio_samples/fma_small/115/115268.mp3' — shape: torch.Size([2, 1323119]), sample rate: 44100
🔁 Resampling from 44100 → 48000 Hz
✂️ Trimming waveform to 30s: 1440130 → 1440000
✅ Embedded /home/ivan/PycharmProjects/MPr/audio_samples/fma_small/115/115268.mp3, vector shape: torch.Size([1, 512])
📂 Loaded '/home/ivan/PycharmProjects/MPr/audio_samples/fma_small/115/115765.mp3' — shape: torch.Size([2, 1321967]), sample rate: 44100
🔁 Resampling from 44100 → 48000 Hz
📏 Padding waveform to 30s: 1438876 → 1440000
✅ Embedded /home/ivan/PycharmProjects/MPr/audio_samples/fma_small/115/115765.mp3, vector shape: torch.Size([1, 512])


Embedding audio files:   2%|▎                   | 5/300 [00:01<00:47,  6.16it/s]

...

🔁 Resampling from 44100 → 48000 Hz
📏 Padding waveform to 30s: 1438876 → 1440000
✅ Embedded /home/ivan/PycharmProjects/MPr/audio_samples/fma_small/066/066538.mp3, vector shape: torch.Size([1, 512])
📂 Loaded '/home/ivan/PycharmProjects/MPr/audio_samples/fma_small/066/066394.mp3' — shape: torch.Size([2, 1321967]), sample rate: 44100
🔁 Resampling from 44100 → 48000 Hz
📏 Padding waveform to 30s: 1438876 → 1440000
✅ Embedded /home/ivan/PycharmProjects/MPr/audio_samples/fma_small/066/066394.mp3, vector shape: torch.Size([1, 512])
📂 Loaded '/home/ivan/PycharmProjects/MPr/audio_samples/fma_small/066/066757.mp3' — shape: torch.Size([2, 1321967]), sample rate: 44100


Embedding audio files:  99%|█████████████████▉| 298/300 [00:30<00:00, 10.08it/s]

🔁 Resampling from 44100 → 48000 Hz
📏 Padding waveform to 30s: 1438876 → 1440000
✅ Embedded /home/ivan/PycharmProjects/MPr/audio_samples/fma_small/066/066757.mp3, vector shape: torch.Size([1, 512])
📂 Loaded '/home/ivan/PycharmProjects/MPr/audio_samples/fma_small/066/066469.mp3' — shape: torch.Size([1, 1323119]), sample rate: 44100
🔁 Resampling from 44100 → 48000 Hz
✂️ Trimming waveform to 30s: 1440130 → 1440000
✅ Embedded /home/ivan/PycharmProjects/MPr/audio_samples/fma_small/066/066469.mp3, vector shape: torch.Size([1, 512])
📂 Loaded '/home/ivan/PycharmProjects/MPr/audio_samples/fma_small/066/066058.mp3' — shape: torch.Size([2, 1321967]), sample rate: 44100
🔁 Resampling from 44100 → 48000 Hz
📏 Padding waveform to 30s: 1438876 → 1440000


Embedding audio files: 100%|██████████████████| 300/300 [00:30<00:00,  9.85it/s]

✅ Embedded /home/ivan/PycharmProjects/MPr/audio_samples/fma_small/066/066058.mp3, vector shape: torch.Size([1, 512])
📂 Loaded '/home/ivan/PycharmProjects/MPr/audio_samples/fma_small/066/066650.mp3' — shape: torch.Size([2, 1323119]), sample rate: 44100
🔁 Resampling from 44100 → 48000 Hz
✂️ Trimming waveform to 30s: 1440130 → 1440000
✅ Embedded /home/ivan/PycharmProjects/MPr/audio_samples/fma_small/066/066650.mp3, vector shape: torch.Size([1, 512])

✅ Embedded 300 tracks.
❌ Failed: 0


In [29]:
import os
import numpy as np

np.save("embeddings/audio_vectors.npy", embeddings)
np.save("embeddings/audio_filenames.npy", np.array(filenames))


In [30]:
import os

print("audio_vectors.npy:", os.path.exists("embeddings/audio_vectors.npy"))
print("audio_filenames.npy:", os.path.exists("embeddings/audio_filenames.npy"))


audio_vectors.npy: True
audio_filenames.npy: True


In [31]:
import numpy as np

vectors = np.load("embeddings/audio_vectors.npy")
filenames = np.load("embeddings/audio_filenames.npy")

print("✅ Vectors shape:", vectors.shape)      # Should be (N, 512)
print("✅ Filenames shape:", filenames.shape)  # Should be (N,)


✅ Vectors shape: (300, 512)
✅ Filenames shape: (300,)
